In [1]:
import warnings  
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
from env import get_db_url
import acquire

pd.set_option("display.max_rows", 2500)


I have some questions for you that I need to be answered before the board meeting Thursday afternoon. I need to be able to speak to the following questions. I also need a single slide that I can incorporate into my existing presentation (Google Slides) that summarizes the most important points. My questions are listed below; however, if you discover anything else important that I didn’t think to ask, please include that as well.


1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
7. Which lessons are least accessed?
8. Anything else I should be aware of?


In [2]:
df = acquire.get_curriculum_data()

In [3]:
df.head()

,date,time,path,user_id,ip,name,start_date,end_date,program_id
0,2018-01-26,09:55:03,/,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
1,2018-01-26,09:56:02,java-ii,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,2.0


In [4]:
df.shape

(900223, 9)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900223 entries, 0 to 900222
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        900223 non-null  object 
 1   time        900223 non-null  object 
 2   path        900222 non-null  object 
 3   user_id     900223 non-null  int64  
 4   ip          900223 non-null  object 
 5   name        847330 non-null  object 
 6   start_date  847330 non-null  object 
 7   end_date    847330 non-null  object 
 8   program_id  847330 non-null  float64
dtypes: float64(1), int64(1), object(7)
memory usage: 68.7+ MB


In [6]:
df['date_time'] = df['date'] + ' ' + df['time']

In [7]:
# convert dates to date time type
df['date_time'] = pd.to_datetime(df['date_time'])
df['start_date'] = pd.to_datetime(df['start_date']).dt.date
df['end_date'] = pd.to_datetime(df['end_date']).dt.date

In [8]:
df = df.set_index('date_time')
df.head()

,date,time,path,user_id,ip,name,start_date,end_date,program_id
date_time,,,,,,,,,
2018-01-26 09:55:03,2018-01-26,09:55:03,/,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
2018-01-26 09:56:02,2018-01-26,09:56:02,java-ii,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
2018-01-26 09:56:05,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
2018-01-26 09:56:06,2018-01-26,09:56:06,slides/object_oriented_programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
2018-01-26 09:56:24,2018-01-26,09:56:24,javascript-i/conditionals,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,2.0


In [9]:
# assigns values in program_id their program name
df.loc[df['program_id'] == 1.0, 'program_id'] = 'Web Development'
df.loc[df['program_id'] == 2.0, 'program_id'] = 'Web Development'
df.loc[df['program_id'] == 4.0, 'program_id'] = 'Web Development'         
df.loc[df['program_id'] == 3.0, 'program_id'] = 'Data Science'

In [10]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
date,900223,1182,2021-03-19,3104,NaN,NaN,NaN,NaN,NaN,NaN,NaN
time,900223,73167,09:03:00,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN
path,900222,2313,/,50313,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_id,900223.0,NaN,NaN,NaN,458.825707,249.296767,1.0,269.0,475.0,660.0,981.0
ip,900223,5531,97.105.19.58,284579,NaN,NaN,NaN,NaN,NaN,NaN,NaN
name,847330,47,Staff,84031,NaN,NaN,NaN,NaN,NaN,NaN,NaN
start_date,847330,44,2014-02-04,92921,NaN,NaN,NaN,NaN,NaN,NaN,NaN
end_date,847330,45,2014-02-04,84031,NaN,NaN,NaN,NaN,NaN,NaN,NaN
program_id,847330,2,Web Development,743918,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
for col in df.columns:
    print(col)
    print(df[col].value_counts())
    print("-------------------------------")

date
2021-03-19    3104
2021-04-12    2446
2021-03-25    2369
2020-09-08    2304
2021-03-16    2298
2020-07-27    2275
2021-02-02    2257
2021-04-05    2236
2021-03-29    2230
2020-07-16    2196
2021-04-15    2192
2021-04-06    2184
2021-03-23    2171
2021-04-19    2156
2020-04-20    2144
2020-09-28    2133
2020-08-03    2117
2021-01-11    2046
2020-10-26    2039
2020-07-28    2031
2020-06-01    1992
2021-03-22    1977
2020-10-22    1976
2021-03-18    1975
2020-09-21    1966
2020-08-10    1944
2020-06-16    1939
2021-04-20    1930
2020-11-02    1919
2020-09-11    1917
2020-12-08    1893
2019-10-07    1879
2021-04-01    1874
2021-01-19    1874
2020-07-24    1852
2020-07-20    1848
2021-03-08    1846
2020-08-06    1843
2021-01-26    1838
2020-11-30    1835
2020-12-14    1825
2021-02-09    1825
2020-06-15    1824
2020-12-10    1814
2021-01-05    1814
2020-05-28    1810
2020-11-19    1797
2020-05-19    1790
2021-04-08    1790
2020-07-21    1785
2020-11-16    1779
2021-01-08    1773
2021-02

97.105.19.58       284579
97.105.19.61        61662
192.171.117.210      9515
71.150.217.33        6791
76.185.145.231       4754
                    ...  
172.58.109.140          1
99.203.213.59           1
99.203.212.173          1
173.239.198.246         1
72.181.124.244          1
Name: ip, Length: 5531, dtype: int64
-------------------------------
name
Staff         84031
Ceres         40730
Zion          38096
Jupiter       37109
Fortuna       36902
Voyageurs     35636
Ganymede      33844
Apex          33568
Deimos        32888
Darden        32015
Teddy         30926
Hyperion      29855
Betelgeuse    29356
Ulysses       28534
Europa        28033
Xanadu        27749
Bayes         26538
Wrangell      25586
Andromeda     25359
Kalypso       23691
Curie         21582
Yosemite      20743
Bash          17713
Luna          16623
Marco         16397
Easley        14715
Lassen         9587
Arches         8890
Florence       8562
Sequoia        7444
Neptune        7276
Olympic        4954


In [12]:
df.isna().sum()

date              0
time              0
path              1
user_id           0
ip                0
name          52893
start_date    52893
end_date      52893
program_id    52893
dtype: int64

**Notes:** 
- What is path relevent to? -why only 1 null?
- Are the 52,893 rows with no name, start_date, end_date, program_id just nulls, or could that be something else?


In [13]:
df[df.name.isna()]

,date,time,path,user_id,ip,name,start_date,end_date,program_id
date_time,,,,,,,,,
2018-01-26 16:46:16,2018-01-26,16:46:16,/,48,97.105.19.61,NaN,NaT,NaT,NaN
2018-01-26 16:46:24,2018-01-26,16:46:24,spring/extra-features/form-validation,48,97.105.19.61,NaN,NaT,NaT,NaN
2018-01-26 17:54:24,2018-01-26,17:54:24,/,48,97.105.19.61,NaN,NaT,NaT,NaN
2018-01-26 18:32:03,2018-01-26,18:32:03,/,48,97.105.19.61,NaN,NaT,NaT,NaN
2018-01-26 18:32:17,2018-01-26,18:32:17,mysql/relationships/joins,48,97.105.19.61,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...
2021-04-21 12:49:00,2021-04-21,12:49:00,javascript-ii,717,136.50.102.126,NaN,NaT,NaT,NaN
2021-04-21 12:49:02,2021-04-21,12:49:02,javascript-ii/es6,717,136.50.102.126,NaN,NaT,NaT,NaN
2021-04-21 12:51:27,2021-04-21,12:51:27,javascript-ii/map-filter-reduce,717,136.50.102.126,NaN,NaT,NaT,NaN


In [14]:
df[df.name.isna()].user_id.value_counts()

354    2965
736    2358
363    2248
716    2136
368    2085
723    1692
349    1663
720    1646
48     1581
727    1573
362    1561
346    1513
355    1436
721    1433
719    1420
353    1398
714    1336
717    1317
715    1159
367    1141
728    1134
366    1095
713     977
351     886
372     867
61      816
352     773
718     753
111     753
361     732
810     709
364     637
357     605
731     528
359     519
722     515
724     477
429     471
58      462
729     461
403     461
726     456
369     448
782     370
88      334
365     331
725     308
79      276
360     251
814     187
86      177
54      146
358     138
375     132
406      90
350      87
73       84
74       80
166      77
317      73
744      69
247      66
100      59
544      55
78       53
103      46
815      46
59       42
64       25
63       23
213      21
356      21
137      10
644       6
663       4
62        4
89        3
176       3
Name: user_id, dtype: int64

In [15]:
df[df.name.isna()].ip.value_counts()

97.105.19.58       15931
70.117.16.60        1903
67.11.117.74        1729
70.94.165.107       1715
23.116.170.48       1616
69.58.159.196       1602
66.25.163.198       1502
70.123.200.120      1320
97.105.19.61        1132
76.185.211.152      1120
104.182.13.78       1111
70.121.134.234      1031
136.50.11.90         961
66.69.62.219         934
173.173.113.51       796
45.23.250.16         741
23.127.174.233       718
70.123.200.67        694
108.208.56.254       622
24.243.64.3          601
24.26.242.9          506
72.181.105.81        483
70.130.121.144       458
99.151.201.126       456
72.177.182.225       433
66.69.98.11          416
72.181.113.170       408
67.11.134.242        405
216.177.189.26       394
107.218.181.206      391
192.171.117.210      391
68.201.219.223       389
136.50.57.27         376
72.190.162.229       349
136.50.23.183        321
70.123.200.145       317
70.120.8.59          312
64.148.245.30        308
216.177.165.194      279
24.251.143.169       271


In [16]:
df[(df.ip=='97.105.19.58')]

,date,time,path,user_id,ip,name,start_date,end_date,program_id
date_time,,,,,,,,,
2018-07-16 07:51:57,2018-07-16,07:51:57,javascript-ii/promises,211,97.105.19.58,Arches,2014-02-04,2014-04-22,Web Development
2018-07-16 07:58:03,2018-07-16,07:58:03,javascript-ii/map-filter-reduce,211,97.105.19.58,Arches,2014-02-04,2014-04-22,Web Development
2018-07-16 07:59:50,2018-07-16,07:59:50,javascript-ii/promises,211,97.105.19.58,Arches,2014-02-04,2014-04-22,Web Development
2018-07-16 08:07:37,2018-07-16,08:07:37,index.html,53,97.105.19.58,Lassen,2016-07-18,2016-11-10,Web Development
2018-07-16 08:07:41,2018-07-16,08:07:41,javascript-ii,53,97.105.19.58,Lassen,2016-07-18,2016-11-10,Web Development
...,...,...,...,...,...,...,...,...,...
2020-03-23 15:10:09,2020-03-23,15:10:09,toc,625,97.105.19.58,Ganymede,2020-03-23,2020-08-20,Web Development
2020-03-23 15:10:18,2020-03-23,15:10:18,toc,623,97.105.19.58,Ganymede,2020-03-23,2020-08-20,Web Development
2020-03-23 15:20:39,2020-03-23,15:20:39,/,627,97.105.19.58,Ganymede,2020-03-23,2020-08-20,Web Development


In [17]:
df[(df.ip=='97.105.19.58') & (df.name=='Staff')]

,date,time,path,user_id,ip,name,start_date,end_date,program_id
date_time,,,,,,,,,
2018-12-06 14:15:29,2018-12-06,14:15:29,appendix/further-reading/java/intellij-tomcat-...,53,97.105.19.58,Staff,2014-02-04,2014-02-04,Web Development
2018-12-06 14:25:40,2018-12-06,14:25:40,javascript-i,314,97.105.19.58,Staff,2014-02-04,2014-02-04,Web Development
2018-12-06 14:25:41,2018-12-06,14:25:41,javascript-i/coffee-project,314,97.105.19.58,Staff,2014-02-04,2014-02-04,Web Development
2018-12-06 15:40:48,2018-12-06,15:40:48,toc,314,97.105.19.58,Staff,2014-02-04,2014-02-04,Web Development
2018-12-06 15:40:55,2018-12-06,15:40:55,html-css,314,97.105.19.58,Staff,2014-02-04,2014-02-04,Web Development
...,...,...,...,...,...,...,...,...,...
2020-03-23 09:51:48,2020-03-23,09:51:48,/,257,97.105.19.58,Staff,2014-02-04,2014-02-04,Web Development
2020-03-23 09:51:54,2020-03-23,09:51:54,toc,257,97.105.19.58,Staff,2014-02-04,2014-02-04,Web Development
2020-03-23 09:51:58,2020-03-23,09:51:58,html-css,257,97.105.19.58,Staff,2014-02-04,2014-02-04,Web Development


In [40]:
df[df.name=='Staff'].ip.value_counts()

97.105.19.58       23593
71.150.217.33       6791
76.185.145.231      4754
70.121.183.95       3840
70.112.179.142      2960
173.173.115.201     2599
76.185.131.226      2428
67.10.174.56        2395
67.11.146.88        2269
52.144.111.131      2232
66.69.86.155        1631
12.106.208.194      1594
70.125.129.152      1538
76.187.230.164      1521
67.11.193.143       1482
72.178.123.235      1274
75.1.166.163        1254
99.88.62.179        1071
70.121.215.53       1061
136.50.63.72        1049
66.69.40.223         989
76.185.155.178       922
65.43.115.179        856
70.115.238.206       850
67.11.120.84         677
24.160.137.86        646
76.192.103.34        556
70.121.199.82        442
173.173.109.203      437
70.118.8.243         419
70.120.31.121        402
72.190.139.120       368
24.243.70.180        360
76.185.152.173       307
67.11.113.196        304
162.250.18.140       298
184.62.220.53        269
73.186.43.188        259
162.233.171.112      253
173.174.218.223      244


In [41]:
df[df.name.isna()].ip.value_counts()

97.105.19.58       15931
70.117.16.60        1903
67.11.117.74        1729
70.94.165.107       1715
23.116.170.48       1616
69.58.159.196       1602
66.25.163.198       1502
70.123.200.120      1320
97.105.19.61        1132
76.185.211.152      1120
104.182.13.78       1111
70.121.134.234      1031
136.50.11.90         961
66.69.62.219         934
173.173.113.51       796
45.23.250.16         741
23.127.174.233       718
70.123.200.67        694
108.208.56.254       622
24.243.64.3          601
24.26.242.9          506
72.181.105.81        483
70.130.121.144       458
99.151.201.126       456
72.177.182.225       433
66.69.98.11          416
72.181.113.170       408
67.11.134.242        405
216.177.189.26       394
107.218.181.206      391
192.171.117.210      391
68.201.219.223       389
136.50.57.27         376
72.190.162.229       349
136.50.23.183        321
70.123.200.145       317
70.120.8.59          312
64.148.245.30        308
216.177.165.194      279
24.251.143.169       271


1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
    - Web Development = javascript-i
    - Data Science = classification/overview

In [19]:
web_dev = df[(df.program_id=='Web Development') & (df.path!= '/')]
ds = df[(df.program_id=='Data Science') & (df.path!= '/')]

In [20]:
web_dev.path.value_counts()

javascript-i                                                                                                                                                                                                                                                               18193
toc                                                                                                                                                                                                                                                                        17580
search/search_index.json                                                                                                                                                                                                                                                   15331
java-iii                                                                                                                                                                             

In [21]:
ds.path.value_counts()

search/search_index.json                                                                                                                                                                                                                                                                                                  2203
classification/overview                                                                                                                                                                                                                                                                                                   1785
1-fundamentals/modern-data-scientist.jpg                                                                                                                                                                                                                                                                                  1655
1-fundamentals/AI-ML-DL-timeline.jpg       

In [22]:
pd.set_option("display.max_rows", 2500)
ds.path.groupby(ds.name).value_counts()

name      path                                                                                                                                                                                                                                                                                                                  
Bayes     1-fundamentals/modern-data-scientist.jpg                                                                                                                                                                                                                                                                                   650
          1-fundamentals/AI-ML-DL-timeline.jpg                                                                                                                                                                                                                                                                                       648
          1-fundament

In [23]:
def value_counts_and_frequencies(s: pd.Series, dropna=True) -> pd.DataFrame:
    return pd.merge(
        s.value_counts(dropna=False).rename('count'),
        s.value_counts(dropna=False, normalize=True).rename('proba'),
        left_index=True,
        right_index=True,
    )


In [24]:
path_ds = value_counts_and_frequencies(ds.path)
path_ds.head()

,count,proba
search/search_index.json,2203,0.023176
classification/overview,1785,0.018779
1-fundamentals/modern-data-scientist.jpg,1655,0.017411
1-fundamentals/AI-ML-DL-timeline.jpg,1651,0.017369
1-fundamentals/1.1-intro-to-data-science,1633,0.017180


In [25]:
path_web_dev = value_counts_and_frequencies(web_dev.path)
path_web_dev.head()

,count,proba
javascript-i,18193,0.025754
toc,17580,0.024886
search/search_index.json,15331,0.021702
java-iii,13162,0.018632
html-css,13111,0.018560


2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [26]:
ds.path.groupby(ds.name).sum()

name
Bayes       3-sql/1-mysql-overview2-storytelling/bad-chart...
Curie       loginloginlogin4-python/1-overview1-fundamenta...
Darden      13-advanced-topics/1-tidy-data1-fundamentals/1...
Easley      python/data-types-and-variablesfundamentals/in...
Florence    fundamentals/intro-to-data-sciencefundamentals...
Name: path, dtype: object

In [27]:
web_dev.path.groupby(web_dev.name).sum()

name
Andromeda     assets/js/pdfmake.min.js.maptoctochtml-csshtml...
Apex          toctoctoctochtml-csshtml-css/introductionjava-...
Apollo        content/html-csscontent/html-css/gitbook/image...
Arches        javascript-ijavascript-i/functionsappendix/fur...
Badlands      prework/fundamentalsprework/fundamentalsprewor...
Bash          html-csshtml-csshtml-csstoctochtml-csshtml-css...
Betelgeuse    html-csstoctoctoctocspringtocappendixhtml-csst...
Ceres         tocquizjavascript-ispringjava-ijavascript-ijav...
Deimos        html-cssjavascript-ihtml-csshtml-csshtml-cssht...
Denali        mkdocs/search_index.jsonprework/databasesprewo...
Europa        toctoctoctoctoctoctoctochtml-csstoctoctocmysql...
Fortuna       tochtml-css/introductionhtml-css/elementstocto...
Franklin      java-iiijava-iii/user-inputjavascript-ijavascr...
Ganymede      java-itoctoctoctochtml-csshtml-css/introductio...
Glacier       prework/fundamentalsprework/databasesprework/d...
Hampton       java-iijava-ii/object

3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [28]:
ds.path.groupby(ds.user_id).value_counts().tail(25)

user_id  path                                                  
949      python/pandas-lesson                                      2
         sql/relationships                                         2
         1-fundamentals/1.1-intro-to-data-science                  1
         1-fundamentals/AI-ML-DL-timeline.jpg                      1
         1-fundamentals/modern-data-scientist.jpg                  1
         basic-statements                                          1
         case-statements                                           1
         classification/prep                                       1
         cli/overview                                              1
         databases                                                 1
         florence-python-assessment.html                           1
         functions                                                 1
         fundamentals                                              1
         fundamentals/data-science-modu

In [29]:
df[df.user_id==949]

,date,time,path,user_id,ip,name,start_date,end_date,program_id
date_time,,,,,,,,,
2021-03-16 11:54:58,2021-03-16,11:54:58,fundamentals,949,64.132.4.102,Florence,2021-03-15,2021-09-03,Data Science
2021-03-16 11:55:18,2021-03-16,11:55:18,fundamentals/intro-to-data-science,949,64.132.4.102,Florence,2021-03-15,2021-09-03,Data Science
2021-03-16 11:55:18,2021-03-16,11:55:18,fundamentals/modern-data-scientist.jpg,949,64.132.4.102,Florence,2021-03-15,2021-09-03,Data Science
2021-03-16 11:55:18,2021-03-16,11:55:18,fundamentals/AI-ML-DL-timeline.jpg,949,64.132.4.102,Florence,2021-03-15,2021-09-03,Data Science
2021-03-16 14:52:06,2021-03-16,14:52:06,fundamentals/intro-to-data-science,949,64.132.4.102,Florence,2021-03-15,2021-09-03,Data Science
2021-03-16 14:52:07,2021-03-16,14:52:07,fundamentals/modern-data-scientist.jpg,949,64.132.4.102,Florence,2021-03-15,2021-09-03,Data Science
2021-03-16 14:52:07,2021-03-16,14:52:07,fundamentals/AI-ML-DL-timeline.jpg,949,64.132.4.102,Florence,2021-03-15,2021-09-03,Data Science
2021-03-16 15:47:09,2021-03-16,15:47:09,fundamentals/intro-to-data-science,949,64.132.4.102,Florence,2021-03-15,2021-09-03,Data Science
2021-03-16 15:47:10,2021-03-16,15:47:10,fundamentals/modern-data-scientist.jpg,949,64.132.4.102,Florence,2021-03-15,2021-09-03,Data Science


In [31]:
web_dev.groupby(web_dev.user_id).value_counts().tail(25)

user_id  date        time      path                                              ip            name     start_date  end_date    program_id     
981      2021-04-21  09:09:39  html-css                                          67.11.98.144  Neptune  2021-03-15  2021-09-03  Web Development    1
                     09:09:42  html-css/css-ii/bootstrap-grid-system             67.11.98.144  Neptune  2021-03-15  2021-09-03  Web Development    1
                     09:29:08  html-css/css-ii/bootstrap-grid-system             67.11.98.144  Neptune  2021-03-15  2021-09-03  Web Development    1
                     10:05:39  examples/css/bootstrap-grids.html                 67.11.98.144  Neptune  2021-03-15  2021-09-03  Web Development    1
                     11:01:19  html-css                                          67.11.98.144  Neptune  2021-03-15  2021-09-03  Web Development    1
         2021-04-19  13:41:12  html-css/css-ii/bootstrap-grid-system             67.11.98.144  Neptune  2021-03